In [32]:
import tensorflow as tf
import os
import scipy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from vindy import SindyNetwork
from vindy.libraries import PolynomialLibrary
from vindy.layers import SindyLayer, VindyLayer
from vindy.distributions import Gaussian, Laplace
from vindy.callbacks import (
    SaveCoefficientsCallback,
)
from vindy.utils import add_lognormal_noise


Now let's train the thing

In [41]:
sindy_type = "vindy"  # "sindy" or "vindy", if you either want a deterministic or probabilistic model for the dynamics, respectively.
model_name = "roessler"
seed = 29 # random seed
random_IC = True # use random initial conditions
random_coeff = True # use random parameters (model noise)
measurement_noise_factor = 0.01 # measurement noise factor
model_noise_factor = 0.1 # model noise factor
n_train = 30 # number of training trajectories
n_test = 4 # number of test trajectories

In [42]:
def generate_directories(model_name, sindy_type, scenario_info, outdir):
    # noise before derivative, model error, seed
    outdir = os.path.join(outdir, f"{model_name}", f"{sindy_type}")
    figdir = os.path.join(outdir, "figures", f"{scenario_info}")
    log_dir = os.path.join(
        outdir,
        '{model_name}', 'log', f'{scenario_info}',
    )
    weights_dir = os.path.join(outdir, "weights", f"{scenario_info}")
    # save figure
    for dir in [outdir, figdir, log_dir, weights_dir]:
        if not os.path.isdir(dir):
            os.makedirs(dir)

    return outdir, figdir, log_dir, weights_dir

In [43]:
scenario_info = f"{sindy_type}_nbd__me_{random_coeff}_{model_noise_factor}_seed_{seed}_noise_{measurement_noise_factor}"
_, _, _, weights_dir = generate_directories(model_name, sindy_type, scenario_info, "results")

In [44]:
def roessler(t, x0, a=0.2, b=0.2, c=5.7):
    x, y, z = x0
    return [-y - z, x + a * y, b + z * (x - c)]

In [45]:
# Roessler system parameters
a = 0.2
b = 0.2
c = 5.7

# initial conditions
x0 = -5
y0 = -5
z0 = 0
ic = [x0, y0, z0]
dim = 3
var_names = ["z_1", "z_2", "z_3"]

# time vector
t0, T, nt = 0, 24, 2000
t = np.linspace(t0, T, nt)
dt = t[1] - t[0]


# Generate initial conditions and parameters
if random_IC:
    np.random.seed(seed)
    x0 = np.concatenate(
        [np.random.normal(ic_, scale=2, size=(n_train + n_test, 1)) for ic_ in ic],
        axis=1,
    )
else:
    x0 = np.repeat(np.array([ic])[:, :, np.newaxis], n_train + 1, axis=2)

if random_coeff:
    np.random.seed(seed)
    a_samples = np.random.normal(a, a * model_noise_factor, size=n_train)
    b_samples = np.random.normal(b, b * model_noise_factor, size=n_train)
    c_samples = np.random.normal(c, c * model_noise_factor, size=n_train)
else:
    a_samples = [a]
    b_samples = [b]
    c_samples = [c]

# Generate data
x = np.array(
    [
        scipy.integrate.odeint(lambda x, t: roessler(t, x, a=a_, b=b_, c=c_), x0_, t)
        for i, (a_, b_, c_, x0_) in enumerate(
            zip(a_samples, b_samples, c_samples, x0[:n_train])
        )
    ]
)

# add measurement noise
x = np.array([add_lognormal_noise(x_, measurement_noise_factor)[0] for x_ in x])
x_test = np.array(
    [
        scipy.integrate.odeint(lambda x, t: roessler(t, x, a=a, b=b, c=c), x0_, t)
        for x0_ in x0[n_train:]
    ]
)

# calculate time derivatives
dxdt = [np.array(np.gradient(x_, dt, axis=0)) for x_ in x]
dxdt_test = [np.array(np.gradient(x_, dt, axis=0)) for x_ in x_test]

In [46]:
# reshape data to fit the model
x_train = np.concatenate(x, axis=0)
dxdt_train = np.concatenate(dxdt, axis=0)
x_test = np.concatenate(x_test, axis=0)
dxdt_test = np.concatenate(dxdt_test, axis=0)

# model parameters
libraries = [
    PolynomialLibrary(2, include_bias=True),
]
dt = t[1] - t[0]

# create sindy layer
layer_params = dict(
    state_dim=x_train.shape[1],
    param_dim=0,
    feature_libraries=libraries,
    second_order=False,
    mask=None,
    kernel_regularizer=tf.keras.regularizers.L1L2(l1=0, l2=0),
)
if sindy_type == "vindy":
    sindy_layer = VindyLayer(
        beta=1e-3,
        priors=Laplace(0.0, 1.0),
        **layer_params,
    )
elif sindy_type == "sindy":
    sindy_layer = SindyLayer(
        **layer_params,
    )
else:
    raise ValueError(f"Unknown SINDy type: {sindy_type}")

# create autoencoder sindy model
model = SindyNetwork(
    sindy_layer=sindy_layer,
    x=x_train,
    l_dz=1e0,
    dt=dt,
    second_order=False,
)
print(type(model))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="huber")

<class 'vindy.networks.sindy_network.SindyNetwork'>
